# 📚 **Мати AI**
Решение для полуфинала [AI Challenge](https://aiijc.com) 2025, направление **"LLM-Преподаватель: Ctrl + Check / Образование"**

> *Mathematics is the art of giving the same name to different things* \
> — Henri Poincaré

![](https://github.com/droyti46/aiijc2025/blob/main/img/stage2/banner.png?raw=true)

## 📒 **Обзор архитектуры**

**TL;DR**

**Нормализуем** решение → **LLM** размечает `<w>…</w>` с протяжкой производных ошибок → **стыкуем** метки с исходником через эквивалентности/регексы/Левенштейн → **парсим** теги в интервалы → **голосуем по символам** (порог = 2) → делаем **постобработку** и формируем **устойчивый `submission.csv`**.

![](https://github.com/droyti46/aiijc2025/blob/main/img/stage2/architecture.png?raw=true)

1. **Нормализация решения (preprocess).** Приводим текст к единому канону по Юникоду, пробелам, математическим знакам и спец-символам.
2. **LLM-разметка.** Генерируем разметку ошибок **тегами** `<w>…</w>` поверх исходного решения (строгий промпт, без комментариев). Делаем несколько прогонов (у нас 8). Причем первые 6 с температурой `0.2`, оставшиеся — с температурой `0.3`
3. **«Умная стыковка» (smart stitcher).** Если LLM слегка изменила текст, переносим её теги на **точные позиции** в исходнике с помощью толерантного сопоставления (регулярки, классы эквивалентности символов, локальный фаззи-поиск по Левенштейну).
4. **Парсер тегов → интервалы.** Конвертируем размеченную строку в список полуинтервалов `[[a, b)]`.
5. **Ансамблирование голосованием по символам.** Сводим результаты из нескольких прогонов/моделей в финальную маску ошибок и обратно в интервалы; порог у нас — `threshold=2`.
6. **Постобработка.** Автоматически дополняем разметку: если последнее и предпоследнее числа в решении совпадают по значению, выделяем оба, чтобы финальный ответ и его повтор в тексте были согласованы.

## 💻 **Решение**

### 🔧 **Установка, импорт зависимостей и базовая настройка**

![](https://github.com/droyti46/aiijc2025/blob/main/img/stage2/dependencies.png?raw=true)

In [ ]:
from IPython.display import clear_output

!pip install --upgrade vllm
!pip install rapidfuzz

clear_output()

Не забудьте перезапустить сеанс после скачивания библиотек

In [ ]:
# Стандартная библиотека
import os
import re
import math
import random
import unicodedata as ud
from collections import Counter
from pathlib import Path
from ast import literal_eval

# Сторонние пакеты для анализа и обработки данных
import pandas as pd
import sympy as sp
import numpy as np
from tqdm.notebook import tqdm, trange

# Юпитерские зависимости
from IPython.display import HTML, display

# Библиотеки для работы с LLM
import torch
from vllm import LLM, SamplingParams

In [ ]:
# Фиксируем сид для стабильности
seed = 42
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)

In [ ]:
# Загружаем табличку из среды
# Надеемся, что она расположена на том же уровне, что и блокнот и называется "test_private_new_without_answer.csv"
df = pd.read_csv('test_private_new_without_answer.csv')
df.head()

,idx,task,solution
0,0,"Докажите: если оператор диагонализируем, то ег...",Ситуация такая: у нас линейный оператор T на к...
1,1,Вычисли $ \displaystyle \lim_{x \rightarrow 8}...,"Смотрю на предел при x, стремящемся к 8, у нас..."
2,2,Пусть \(1\le p\le\infty\). Требуется установит...,Хотим показать: если в ℓ^p (при 1 ≤ p ≤ ∞) взя...
3,3,Вычисли $ \displaystyle \lim_{x \rightarrow -1...,"Посмотрим, что делает эта дробь при x, стремящ..."
4,4,Вычисли $ \displaystyle \lim_{x \rightarrow 8}...,"Смотрю на выражение при x, стремящемся к 8: вв..."


In [ ]:
# На всякий случай обрезаем до 400 строк
df = df[:400]
print(f'Размер тестовой выборки: {len(df)}')

Размер тестовой выборки: 400


**⚠️ Внимание**: мы настоятельно рекомендуем проверяющим подключить Google  диск, если вы запускаете этот код в **Google Colab** \
Это ни на что не повлияет: код написан так, что будет работать независимо от директории. \
Однако у нас бывали случаи, когда Colab вылетал после 8 часов работы, а у нас совсем не было сохранений. \
Поэтому при подключении Google диска удобно будут сохраняться чекпоинты и вы сможете сберечь лишние нервы =) \
После вылета Colab нужно всего-лишь снова прожать все ячейки до генерации \
В идеале вам понадобится 2-3 прогона (наш инференс работает `≈12-16 часов` на `A100`) \
Есть возможность не подключать диск, тогда все чекпоинты будут сохранены в среду́. Но в таком случае их придется скачивать и снова вручную загружать при вылете Колаба

In [ ]:
from google.colab import drive
drive.mount("/content/drive")

# Путь к целевой папке на Диске
base_dir = Path("/content/drive/MyDrive")
target_dir = base_dir / 'AI_Challenge_snr/checkpoints'

# Создаём папку (и все недостающие родительские директории)
target_dir.mkdir(parents=True, exist_ok=True)

%cd /content/drive/MyDrive/AI_Challenge_snr/checkpoints

Mounted at /content/drive
/content/drive/MyDrive/AI_Challenge_snr/checkpoints


In [ ]:
#@title Функция для красивого отображения решений с ошибками

def highlight_errors(solution: str, error_segments: list[list[int]]) -> None:
    """
    Отображает текст с подсвеченными ошибками в HTML формате.

    Args:
        solution: строка с решением
        error_segments: список пар [start, end] с позициями ошибок
    """
    # Сортируем сегменты ошибок по начальной позиции
    sorted_segments = sorted(error_segments, key=lambda x: x[0])

    # Создаём HTML с подсветкой
    html_parts = []
    last_pos = 0

    for start, end in sorted_segments:
        # Добавляем текст до ошибки
        if start > last_pos:
            html_parts.append(escape_html(solution[last_pos:start]))

        # Добавляем текст ошибки с подсветкой
        error_text = escape_html(solution[start:end])
        html_parts.append(f'<span class="error">{error_text}</span>')

        last_pos = end

    # Добавляем оставшийся текст
    if last_pos < len(solution):
        html_parts.append(escape_html(solution[last_pos:]))

    # Собираем финальный HTML
    html_content = ''.join(html_parts)

    # CSS стили
    styles = """
    <style>
        .solution-container {
            font-family: 'Georgia', 'Times New Roman', serif;
            font-size: 16px;
            line-height: 1.8;
            padding: 20px;
            background-color: #ffffff;
            color: #000000;
            border-radius: 8px;
            box-shadow: 0 2px 8px rgba(0,0,0,0.1);
            white-space: pre-wrap;
            word-wrap: break-word;
            width: 100%;
            box-sizing: border-box;
        }
        .error {
            background-color: #ffe6e6;
            border: 2px solid #ff4444;
            border-radius: 3px;
            padding: 2px 4px;
            margin: 0 1px;
            color: #000000;
            box-shadow: 0 0 3px rgba(255, 68, 68, 0.3);
            transition: all 0.2s;
        }
        .error:hover {
            background-color: #ffcccc;
            box-shadow: 0 0 6px rgba(255, 68, 68, 0.5);
        }
    </style>
    """

    full_html = f'{styles}<div class="solution-container">{html_content}</div>'

    display(HTML(full_html))

def escape_html(text: str) -> str:
    """Экранирует HTML специальные символы"""
    return (text
            .replace('&', '&amp;')
            .replace('<', '&lt;')
            .replace('>', '&gt;')
            .replace('"', '&quot;')
            .replace("'", '&#39;'))

### 🧹 **Этап 1: Препроцессинг (нормализация)**

**Идея**

Маленькие LLM чувствительны к качеству данных; любые невидимые «артефакты» (неразрывные пробелы, «длинные» тире, математический минус, нестандартные кавычки/деление/умножение) ломают ее и из-за этого она отвечает хуже. Помимо этого мы заметили некоторые приколы, например, в решениях используется **10 разных пробелов** или могут стоять **2 пробела подряд**. Все это мы учитываем

Мы нормализуем все символы и подготавливаем данные к последующей подаче в LLM

In [ ]:
#@title Функция нормализации

# Карта символов для приведения текста к единому виду
# (замена неразрывных пробелов, тире, кавычек, математических символов и т.д.)
CANDIDATE_MAPPING = {
    "\u00A0": " ","\u2002": " ","\u2003": " ","\u2004": " ","\u2005": " ","\u2006": " ",
    "\u2007": " ","\u2008": " ","\u2009": " ","\u200A": " ","\u202F": " ","\u205F": " ","\u3000": " ",
    "\u200B": "","\u200C": "","\u200D": "","\u2060": "","\uFEFF": "",
    "\u2212": "-","\u2010": "-","\u2011": "-","\u2012": "-","\u2013": "-","\u2014": "-","\u2015": "-","\u2043": "-",
    "\u2018": "'","\u2019": "'","\u201A": "'","\u201B": "'","\u201C": '"',"\u201D": '"',"\u201E": '"',"\u201F": '"',
    "\u00D7": "*","\u22C5": "*","\u2219": "*","\u2215": "/","\u2044": "/","\u00F7": "/","\u2217": "*",
    "\u27F6": "→","\u2192": "→","\u21D2": "⇒","\u03BC": "μ","\u00B5": "μ",
    "\u2264": "<=","\u2265": ">=","\u2260": "!=","\u2261": "==","\u221E": "∞",
}

# Строим карту только по реально встречающимся символа
def build_present_mapping(series):
    """
    Формирует карту замен только по реально встречающимся в данных символам.
    Это ускоряет нормализацию и уменьшает размер таблицы трансляции.

    Args:
        series (pd.Series): Столбец pandas со строками, в которых нужно провести замену символов.

    Returns:
        dict: Словарь сопоставлений {символ: замена}, отфильтрованный по реально найденным символам.
    """
    # Объединяем все строки в одну и считаем встречаемость символов
    big = "".join(series.dropna().astype(str).tolist())
    counts = Counter(big)

    # Оставляем только символы, которые реально присутствуют в данных
    return {k: v for k, v in CANDIDATE_MAPPING.items() if k in counts}

# Создаём функцию нормализации
def make_normalizer(present_mapping):
    """
    Создаёт функцию нормализации текста на основе переданной карты символов.

    Возвращает функцию, которая:
    - приводит текст к Unicode-NFKC;
    - заменяет нестандартные символы;
    - унифицирует переводы строк;
    - схлопывает и обрезает пробелы.

    Args:
        present_mapping (dict): Словарь замен символов (например, из build_present_mapping).

    Returns:
        function: Функция normalize_text(s), нормализующая переданную строку.
    """
    def normalize_text(s, to_nfkc=True, collapse_spaces=True, strip_spaces=True):
        """
        Нормализует отдельную строку текста.

        Args:
            s (str): Входная строка.
            to_nfkc (bool, optional): Приводить ли строку к Unicode NFKC. По умолчанию True.
            collapse_spaces (bool, optional): Схлопывать ли повторяющиеся пробелы. По умолчанию True.
            strip_spaces (bool, optional): Удалять ли пробелы по краям строк и строк перевода. По умолчанию True.

        Returns:
            str: Нормализованная строка (или исходное значение, если не str).
        """
        if not isinstance(s, str):
            return s

        # Приведение к канонической Unicode-форме
        out = ud.normalize("NFKC", s) if to_nfkc else s

        # Замена символов по карте
        if present_mapping:
            out = out.translate(str.maketrans(present_mapping))

        # Унификация переводов строк
        out = out.replace("\r\n", "\n").replace("\r", "\n")

        # Схлопывание пробелов и табов
        if collapse_spaces:
            out = re.sub(r"[ \t\u00A0]+", " ", out)

        # Удаление пробелов по краям строк
        if strip_spaces:
            out = "\n".join(line.strip() for line in out.split("\n"))

        return out

    return normalize_text

In [ ]:
# Строим карту замен только по символам, реально встречающимся в решениях
present_mapping = build_present_mapping(df["solution"])

# Создаём функцию нормализации на основе найденных замен
normalize_text = make_normalizer(present_mapping)

# Применяем нормализацию ко всем решениям
df["preprocessed_solution"] = df["solution"].astype(str).map(normalize_text)
df.head()

,idx,task,solution,preprocessed_solution
0,0,"Докажите: если оператор диагонализируем, то ег...",Ситуация такая: у нас линейный оператор T на к...,Ситуация такая: у нас линейный оператор T на к...
1,1,Вычисли $ \displaystyle \lim_{x \rightarrow 8}...,"Смотрю на предел при x, стремящемся к 8, у нас...","Смотрю на предел при x, стремящемся к 8, у нас..."
2,2,Пусть \(1\le p\le\infty\). Требуется установит...,Хотим показать: если в ℓ^p (при 1 ≤ p ≤ ∞) взя...,Хотим показать: если в l^p (при 1 <= p <= ∞) в...
3,3,Вычисли $ \displaystyle \lim_{x \rightarrow -1...,"Посмотрим, что делает эта дробь при x, стремящ...","Посмотрим, что делает эта дробь при x, стремящ..."
4,4,Вычисли $ \displaystyle \lim_{x \rightarrow 8}...,"Смотрю на выражение при x, стремящемся к 8: вв...","Смотрю на выражение при x, стремящемся к 8: вв..."


Мы нормализовали все решения в колонке `solution`. Можно смело двигаться дальше!

### 🔮 **Этап 2: Предсказание 8 ответов LLM с разной температурой**

Ключевой промпт заставляет модель **вернуть исходный текст без изменений**, добавив **только теги** `<w>...</w>` вокруг ошибок, и задаёт строгие правила

In [ ]:
key_prompt = """# РОЛЬ
Строгий математический проверяющий.

# ЗАДАЧА
Найди все ошибки и отметь все производные от них ошибки.

# ВЫВОД
Верни исходный текст решения задачи без изменений, добавив только теги <w>...</w> вокруг ошибочных фрагментов. Никаких комментариев.

# ПРАВИЛА
1) Размечай минимальный неверный фрагмент (число, знак, слово или часть выражения). Пример: если фраза "они получаются вещественными" неверна, выдели только ключевое слово <w>вещественными</w>.
2) Если ошибка логически тянет за собой другие (производные) — пометь их тоже, даже если нарушается минимальность. Выделить производные ошибки важнее, чем соблюдать минимальность.
3) Если левая часть уравнения или преобразования содержит помеченный фрагмент, пометь и правую часть равенства/преобразования (число, выражение, десятичную запись, округление), даже если вычисление само по себе верное.
4) Если число/символ помечено, пометь все его дальнейшие появления и все числовые результаты, полученные из него.
5) Разложения и тождества: помечай любой лишний или неправильно разложенный член (например, в тригонометрических формулах) и все последствия от неправильного разложения.
6) Нормализация степеней/масштабов: при выносе и замене помечай неверные степени и все зависящие от них результаты.
7) Пространство исходов/мощности: если неверно выбран базовый счёт (например, «по m вариантов»), пометь и производные мощности/значения (m^3, итоги).
8) Не меняй форматирование, пробелы, переносы строк и сам текст.
9) Не выделяй введение обозначений. То есть, если x ошибочен, и написано "пусть a = x / 2", мы не выделяем эту запись тегами. Выделяй такие записи, только если в них уже производится численная подстановка или логически неверное равенство.
Пример:
"2^1 + 2^18 = <w>2^20</w>. А 2^20 = 1048576, поэтому ответ: <w>1048576</w>".

# ЧЕК-ЛИСТ
1) Найди первичные ошибки.
2) Построй зависимости (что из чего получено). Подумай для каждого числа и для каждой формулы, стоит её помечать или нет (производная она ошибка или нет).
3) Протащи метки на все зависимые числа/выражения/округления и их повторные упоминания.

# ИЛЛЮСТРАЦИИ ФОРМАТА
Было:  2+2=5.
Стало: 2+2=<w>5</w>.

Было:  sin(a+b) = sin a cos b + cos a sin b + sin a sin b.
Стало:  sin(a+b) = sin a cos b + cos a sin b + <w>sin a sin b</w>.

Было:  x = 3. Тогда, возведём x в квадрат: x^2 = 16.
Стало:  x = 3. Тогда, возведём x в квадрат: x^2 = <w>16</w>.

# ДАННЫЕ
task:
{task}

solution:
{solution}
"""

**💡 Возможные ошибки к ячейке ниже**

❌ **`OSError: [Errno 107] Transport endpoint is not connected: 'Qwen/Qwen3-4B-Thinking-2507'`**: \
Странная, не зависящая от нас ошибка из-за смонтированного `Google диска` и `vLLM`. Исправляется, если перезапустить среду \
**Исправление:** `Среда выполнения → Перезапустить сеанс`

In [ ]:
# Используем модель Qwen3-4B-Thinking для нахождения ошибок в решениях
# Модель показывает хорошие результаты на математических задачах и умеет рассуждать пошагово
model_path = "Qwen/Qwen3-4B-Thinking-2507"

# Инициализация LLM с заданными параметрами
llm = LLM(
    model=model_path,
    trust_remote_code=True,
    gpu_memory_utilization=0.95,
    max_model_len=50_000
)

In [ ]:
# Подготовка данных для диалогов модели
conversations = []  # Список диалогов формата [[{"role": ..., "content": ...}], ...]
all_batch_idxs = [] # Храним индексы строк, чтобы потом сопоставить с ответами модели

# Формируем список промтов для каждого решения
for idx in trange(len(df)):
    # Текст задания
    task = df.loc[idx, 'task']
    # Решение пользователя (предобработанное)
    solution = df.loc[idx, 'preprocessed_solution']

    # Подставляем задание и решение в шаблон промта
    conversations.append([
        {
            "role": "user",
            "content": key_prompt.replace('{task}', task).replace('{solution}', solution)
        }
    ])

    # Запоминаем индекс, чтобы потом объединить результаты
    all_batch_idxs.append(idx)

In [ ]:
# Для проверки выведем первый сформированный промпт
print(conversations[0][0]['content'])

# РОЛЬ
Строгий математический проверяющий.

# ЗАДАЧА
Найди все ошибки и отметь все производные от них ошибки.

# ВЫВОД
Верни исходный текст решения задачи без изменений, добавив только теги <w>...</w> вокруг ошибочных фрагментов. Никаких комментариев.

# ПРАВИЛА
1) Размечай минимальный неверный фрагмент (число, знак, слово или часть выражения). Пример: если фраза "они получаются вещественными" неверна, выдели только ключевое слово <w>вещественными</w>.
2) Если ошибка логически тянет за собой другие (производные) — пометь их тоже, даже если нарушается минимальность. Выделить производные ошибки важнее, чем соблюдать минимальность.
3) Если левая часть уравнения или преобразования содержит помеченный фрагмент, пометь и правую часть равенства/преобразования (число, выражение, десятичную запись, округление), даже если вычисление само по себе верное.
4) Если число/символ помечено, пометь все его дальнейшие появления и все числовые результаты, полученные из него.
5) Разложения и тождества: пом

In [ ]:
# Настройки сэмплирования для генерации
# Более консервативная и устойчивая генерация с температурой 0.2
sampling_params_conservative = SamplingParams(
    n=1,
    temperature=0.2,
    max_tokens=45000,
    seed=seed
)

# Более кративная и разнообразная генерация с температурой 0.3
sampling_params_creative = SamplingParams(
    n=1,
    temperature=0.3,
    max_tokens=45000,
    seed=seed
)

In [ ]:
def parse_outputs_into_df(df: pd.DataFrame, outputs, all_batch_idxs) -> pd.DataFrame:
    """
    Парсит ответы модели и добавляет их в датафрейм.

    Args:
        df: Исходный датафрейм.
        outputs: Список ответов от llm.chat, упорядоченный в соответствии с conversations.
        all_batch_idxs: Индексы строк df, соответствующие каждому запросу.

    Returns:
        Копия df с добавленными колонками response_0, response_1, ...
    """
    new_df = df.copy()

    # Разбор ответов модели и запись их в датафрейм
    for idx, output in tqdm(zip(all_batch_idxs, outputs), total=len(outputs), desc="Parsing model outputs"):
        for i, candidate in enumerate(output.outputs):
            response = candidate.text.strip()
            new_df.at[idx, f"response_{i}"] = response

    return new_df

In [ ]:
#@title Проверка на существование чекпоинтов

# Сколько всего прогонов планируется
N = 8

# Папка с чекпоинтами (по умолчанию — текущая рабочая директория)
ckpt_dir = Path.cwd()

# Паттерн имени чекпоинта: df_{i}.csv
name_re = re.compile(r"^df_(\d+)\.csv$")

# Соберём индексы существующих чекпоинтов
indices = []
for p in ckpt_dir.iterdir():
    m = name_re.match(p.name)
    if m:
        indices.append(int(m.group(1)))

if indices:
    i_last = max(indices)
    print(f"Найдено чекпоинтов: {len(indices)}")
    print("Список:", sorted(indices))
    print(f"Последний чекпоинт: df_{i_last}.csv")
else:
    i_last = -1
    print("Чекпоинтов не найдено. Начинаем с нуля.")

# Сколько осталось прогонов по формуле: N - i - 1 (где i — номер последнего чекпоинта)
remaining = max(0, N - i_last - 1)
print(f"Осталось прогонов: {remaining}")

# Стартовый индекс для цикла
start = i_last + 1

Найдено чекпоинтов: 8
Список: [0, 1, 2, 3, 4, 5, 6, 7]
Последний чекпоинт: df_7.csv
Осталось прогонов: 0


In [ ]:
if start >= N:
    raise SystemExit("Все прогоны уже выполнены. Нечего дорабатывать.")

# Запуск модели в режиме чат-инференса
# conversations — список диалогов
for i in range(start, N):
    # Используем креативную генерацию, если осталось сгенерировать 2 и меньше ответов
    # и консервативную в ином случае
    sampling_params = sampling_params_creative if i >= 6 else sampling_params_conservative
    print(f'Мати AI | 💡 Начинается генерация ответа с температурой {sampling_params.temperature}')
    outputs = llm.chat(
        conversations,
        sampling_params=sampling_params,
        use_tqdm=True,
    )

    # Парсим ответы в датафрейм и сохраняем
    df_parsed = parse_outputs_into_df(df, outputs, all_batch_idxs)
    df_parsed.to_csv(f'df_{i}.csv', index=False)

    print(f'Мати AI | ✅ Сохранен чекпоинт {i}')

### 🧱 **Этап 3: Стыковка и парсинг**

Даже при строгом промпте LLM может **незначительно изменить** пробелы/символы, вырезать некоторые слово или, наоборот, вставить свои. Чтобы не потерять точные позиции, мы переносим `<w>…</w>` со сгенерированного LLM `marked_solution` **на исходный `solution`** с аккуратным сопоставлением. Помимо этого надо учитывать, что после нормализации мы убрали все двойные пробелы и этот сдвиг тоже надо учитывать. **Стыковка** — это сердце пайплайна.

![](https://github.com/droyti46/aiijc2025/blob/main/img/stage2/stitch.png?raw=true)

**Классы эквивалентности**

* Строим двунаправленные отображения: `src→dst` (нормализатор) и `dst→{варианты}` (класс эквивалентов).
* Эквивалентны: все виды пробелов (кроме `\n`), минусы, кавычки, умножение/деление, `→/⇒`, `μ`, `<=/>=/!=/==`, `∞` и др.
* На основе классов строится **класс пробелов** `[_WS_CLASS]` и регексы для **блоков пробелов** без разрыва строк.

**Толерантный паттерн для поиска**

`_space_and_equiv_aware_pattern(pat)` превращает фрагмент из `<w>…</w>` в мощный регекс, который:

* **склеивает** блоки пробелов как `(?:WS+)`;
* **игнорирует** `$…$` LaTeX-ограничители;
* понимает **LaTeX-макросы** и сопоставляет им **юнкодовские аналоги** (например, `\sigma` ↔ `σ`), а ещё **микро-пробелы** `\, \; \: \!`;
* расширяет многосимвольные токены (`<=, >=, !=, ==`) альтернативами всех эквивалентов;
* для одиночных символов формирует **класс** из всех эквивалентных вариантов.

**Стратегия «лучшего совпадения»**

`_best_fuzzy_span` выдаёт `{start, end, score}` и идёт лесенкой от точных к «мягким» совпадениям:

1. глобальный `find`;
2. глобальный **толерантный** поиск по регексу;
3. локальный `find` в окне;
4. локальный толерантный;
5. **фаззи-сравнение** (RapidFuzz Levenshtein) с адаптивным порогом по длине паттерна.
   Параллельно выбираются **якоря** (начало паттерна, длинные «словесные» куски, кластеры знаков) и уточняется старт, чтобы не «съезжать» на грани слова.

**Стыковка последовательности меток**

`stitch_markers_fast(original, generated)` идёт по всем `<w>…</w>` подряд, следит за «соседством» (эвристика `neighbor_margin`), вставляет **ровно оригинальные подстроки** между `<w>` и `</w>`. Это сохраняет исходные пробелы и «редкие» символы **без пересчёта индексов**.

---

**Парсинг тегов в интервалы**

`extract_tag_segments(text_with_tags)` считывает `<w>`/`</w>`, **игнорируя сами теги при подсчёте длины** и корректно обрабатывая буквальный символ `<` в тексте. На выходе — список **полуинтервалов** `[[start, end)]` в координатах **чистого решения без тегов**.

In [ ]:
# Загрузим ответы всех моделей в ансамбле
dfs = []
for i in range(8):
    # На всякий случай обрежем до 400 строк
    dfs.append(
        pd.read_csv(f'df_{i}.csv')[:400]
    )

In [ ]:
# Посмотрим на ответы первой модели
dfs[0].head()

,idx,task,solution,preprocessed_solution,response_0
0,0,"Докажите: если оператор диагонализируем, то ег...",Ситуация такая: у нас линейный оператор T на к...,Ситуация такая: у нас линейный оператор T на к...,"Хорошо, мне нужно проверить решение задачи и н..."
1,1,Вычисли $ \displaystyle \lim_{x \rightarrow 8}...,"Смотрю на предел при x, стремящемся к 8, у нас...","Смотрю на предел при x, стремящемся к 8, у нас...","Хорошо, давай разберусь с этой задачей. Мне ну..."
2,2,Пусть \(1\le p\le\infty\). Требуется установит...,Хотим показать: если в ℓ^p (при 1 ≤ p ≤ ∞) взя...,Хотим показать: если в l^p (при 1 <= p <= ∞) в...,"Хорошо, мне нужно проверить решение задачи о п..."
3,3,Вычисли $ \displaystyle \lim_{x \rightarrow -1...,"Посмотрим, что делает эта дробь при x, стремящ...","Посмотрим, что делает эта дробь при x, стремящ...","Хорошо, давай разберусь с этой задачей. Мне ну..."
4,4,Вычисли $ \displaystyle \lim_{x \rightarrow 8}...,"Смотрю на выражение при x, стремящемся к 8: вв...","Смотрю на выражение при x, стремящемся к 8: вв...","Хорошо, давай разберусь с этой задачей. Мне ну..."


In [ ]:
#@title Большая, умная, сложная стыковка

import re
import unicodedata as ud
from functools import lru_cache

# =========================
#   ЛЕВЕНШТЕЙН (быстрый)
# =========================
try:
    from rapidfuzz.distance import Levenshtein as RFLev

    def lev(a, b):
        """Нормированное расстояние [0..1]."""
        if a == b:
            return 0.0
        d = RFLev.distance(a, b)
        return d / max(len(a), len(b))

except Exception:
    import difflib

    def lev(a, b):
        """Запасной путь через difflib."""
        if a == b:
            return 0.0
        ratio = difflib.SequenceMatcher(None, a, b).ratio()
        return 1.0 - ratio


# =========================================
#   ЭКВИВАЛЕНТНОСТИ СИМВОЛОВ / ПРОБЕЛОВ
# =========================================

_DEFAULT_CANDIDATE_MAPPING = {
    "\u00A0": " ","\u2002": " ","\u2003": " ","\u2004": " ","\u2005": " ","\u2006": " ",
    "\u2007": " ","\u2008": " ","\u2009": " ","\u200A": " ","\u202F": " ","\u205F": " ","\u3000": " ",
    "\u200B": "","\u200C": "","\u200D": "","\u2060": "","\uFEFF": "",
    "\u2212": "-","\u2010": "-","\u2011": "-","\u2012": "-","\u2013": "-","\u2014": "-","\u2015": "-","\u2043": "-",
    "\u2018": "'","\u2019": "'","\u201A": "'","\u201B": "'","\u201C": '"',"\u201D": '"',"\u201E": '"',"\u201F": '"',
    "\u00D7": "*","\u22C5": "*","\u2219": "*","\u2215": "/","\u2044": "/","\u00F7": "/","\u2217": "*",
    "\u27F6": "→","\u2192": "→","\u21D2": "⇒","\u03BC": "μ","\u00B5": "μ",
    "\u2264": "<=","\u2265": ">=","\u2260": "!=","\u2261": "==","\u221E": "∞",
}

# Глобальные конфиги (зависят от present_mapping)
_FORWARD_MAP = None     # dict[src->dst]
_EQUIV = None           # dict[dst-> set(src)] — для поиска по оригиналу
_WS_CLASS = None        # строка "[...]" без \n
_WS_RX = None           # re: блоки пробелов (не включая \n)

def _invert_mapping(candidate_mapping: dict) -> dict[str, set[str]]:
    """dst -> {src}, добавляя сам dst в класс; базовые одиночные токены — по умолчанию."""
    inv: dict[str, set[str]] = {}
    for src, dst in candidate_mapping.items():
        inv.setdefault(dst, set()).add(src)
    for dst in list(inv):
        inv[dst].add(dst)
    for base in [" ", "-", "'", '"', "*", "/", "→", "⇒", "μ", "<=", ">=", "!=", "==", "∞"]:
        inv.setdefault(base, set()).add(base)
    return inv

def configure_equivalences(present_mapping: dict | None = None):
    """Позови ПЕРЕД первым матчем."""
    global _FORWARD_MAP, _EQUIV, _WS_CLASS, _WS_RX
    cm = present_mapping or _DEFAULT_CANDIDATE_MAPPING
    _FORWARD_MAP = cm.copy()
    _EQUIV = _invert_mapping(cm)

    # класс пробелов: всё, что маппится в " ", + обычный пробел и таб (НО НЕ \n)
    ws_chars = set([" ", "\t"])
    ws_chars |= _EQUIV.get(" ", set())
    cc = "".join(sorted({re.escape(c) for c in ws_chars}))
    _WS_CLASS = "[" + cc + "]"
    _WS_RX = re.compile(rf"(?:{_WS_CLASS}+)", flags=0)

    # сбрасываем кэши
    try: _space_aware_rx.cache_clear()
    except Exception: pass
    try: _best_fuzzy_span.cache_clear()
    except Exception: pass

configure_equivalences()  # дефолт


# =========================================
#   НОРМАЛИЗАЦИЯ ДЛЯ ФАЗЗИ-СКОРА
# =========================================
def _norm(s: str) -> str:
    r"""
    Мягкая нормализация:
    - NFKC;
    - translate по FORWARD_MAP (src->dst);
    - \r\n/\r -> \n;
    - схлопнуть блоки «наших» пробелов в один обычный пробел;
    - casefold + trim.
    """
    if not isinstance(s, str):
        return ""
    out = ud.normalize("NFKC", s)
    if _FORWARD_MAP:
        out = out.translate(str.maketrans(_FORWARD_MAP))
    out = out.replace("\r\n", "\n").replace("\r", "\n")
    out = _WS_RX.sub(" ", out) if _WS_RX else re.sub(r"[ \t\u00A0]+", " ", out)
    return out.casefold().strip()


# =========================================
#   ВЫТЯГИВАНИЕ СПАНОВ ИЗ <w>…</w>
# =========================================
def _extract_spans(generated_text: str):
    r"""Читает корректные спаны, лечит склейки </w><w>, выкидывает пустяки."""
    gt = re.sub(r"</w>\s*<w>", " ", generated_text)
    spans = []
    for m in re.finditer(r"<w>(.*?)</w>", gt, flags=re.DOTALL):
        inner = re.sub(r"</?w>", "", m.group(1)).strip()
        if _norm(inner):
            spans.append(inner)
    return spans


# =========================================
#   LaTeX → Unicode (минимально нужное)
# =========================================
_LATEX_TO_UNI = {
    "sum": "∑", "prod": "∏",
    "sigma": "σ", "Sigma": "Σ",
    "tau": "τ", "Tau": "Τ",
    "in": "∈",
    # можно пополнять по надобности
}

_SPACING_MACROS = {",", ";", ":", " ", "!"}  # \, \; \: \<space> \!

# =========================================
#   ПОСТРОИТЕЛЬ ТОЛЕРАНТНОГО ПАТТЕРНА
# =========================================
def _space_and_equiv_aware_pattern(pat: str) -> str:
    r"""
    Строит regex по ОРИГИНАЛУ:
    - блоки пробелов в pat -> (?:WS+)
    - игнорирует $ из LaTeX-матмода
    - \macro с известной заменой -> (?:\\macro|<Юникод>)
    - \, \; \: \<space> -> (?:буквальный макрос | WS+)
      \! -> (?:буквальный \! | пусто) — часто «узкий отрицательный пробел»
    - многосимвольные токены из EQUIV (например "<=") -> альтернация по эквивалентам
    - одиночные символы с эквивалентами -> класс/альтернация
    - прочее — re.escape
    """
    out = []
    i = 0
    multi_tokens = []
    if _EQUIV:
        multi_tokens = sorted([t for t in _EQUIV.keys() if len(t) > 1],
                              key=len, reverse=True)

    N = len(pat)
    while i < N:
        # 1) Блок пробелов
        m = _WS_RX.match(pat, i)
        if m:
            out.append(rf"(?:{_WS_CLASS}+)")
            i = m.end()
            continue

        ch = pat[i]

        # 2) Игнорируем LaTeX-доллары
        if ch == "$":
            i += 1
            continue

        # 3) Многосимвольные токены по эквивалентностям (<=, >=, !=, ==, …)
        matched_multi = False
        for tok in multi_tokens:
            if pat.startswith(tok, i):
                alts = sorted(_EQUIV.get(tok, {tok}), key=len, reverse=True)
                out.append("(?:%s)" % "|".join(re.escape(a) for a in alts))
                i += len(tok)
                matched_multi = True
                break
        if matched_multi:
            continue

        # 4) LaTeX-макросы
        if ch == "\\" and i + 1 < N:
            nxt = pat[i + 1]
            # 4a) \word...
            if nxt.isalpha():
                j = i + 2
                while j < N and pat[j].isalpha():
                    j += 1
                name = pat[i + 1:j]
                # известный греческий/операционный макрос
                if name in _LATEX_TO_UNI:
                    uni = _LATEX_TO_UNI[name]
                    out.append("(?:%s|%s)" % (re.escape("\\" + name), re.escape(uni)))
                    i = j
                    continue
                else:
                    # неизвестный макрос — матчим буквально (на всякий случай)
                    out.append(re.escape("\\" + name))
                    i = j
                    continue
            # 4b) \, \; \: \<space> \! и т.п.
            if nxt in _SPACING_MACROS:
                literal = "\\" + nxt
                if nxt == "!":
                    out.append("(?:%s|)" % re.escape(literal))         # \! или пусто
                else:
                    out.append("(?:%s|%s+)" % (re.escape(literal), _WS_CLASS))  # букв. или пробелы
                i += 2
                continue
            # 4c) прочие двухсимвольные \X — матчим буквально
            out.append(re.escape("\\" + nxt))
            i += 2
            continue

        # 5) Одиночный символ с эквивалентами
        if _EQUIV and ch in _EQUIV:
            eq = _EQUIV[ch]
            single = [c for c in eq if len(c) == 1]
            multi = [c for c in eq if len(c) > 1]
            parts = []
            if single:
                parts.append("[" + "".join(sorted({re.escape(c) for c in single})) + "]")
            parts += [re.escape(c) for c in sorted(multi, key=len, reverse=True)]
            out.append(parts[0] if len(parts) == 1 else "(?:%s)" % "|".join(parts))
            i += 1
            continue

        # 6) По умолчанию — буквальный символ
        out.append(re.escape(ch))
        i += 1

    return "".join(out) or r"(?:)"

def _is_short_alnum_token(pat: str) -> bool:
    s = _norm(pat)
    return 1 <= len(s) <= 2 and s.isalnum()

def _is_word_char(ch: str) -> bool:
    return bool(ch) and (ch.isalnum() or ch == "_")

def _inside_word(text: str, s: int, e: int) -> bool:
    left  = text[s-1] if s > 0 else ""
    right = text[e]   if e < len(text) else ""
    return _is_word_char(left) and _is_word_char(right)


@lru_cache(maxsize=4096)
def _space_aware_rx(pat: str) -> re.Pattern:
    core = _space_and_equiv_aware_pattern(pat)
    if _is_short_alnum_token(pat):
        # запрещаем матч «внутри» слова
        core = r"(?<!\w)" + core + r"(?!\w)"
    return re.compile(core, flags=re.DOTALL)


def _space_aware_search(text: str, start_pos: int, pattern: str):
    """Ищет pattern по оригиналу, учитывая пробелы/эквиваленты/LaTeX."""
    if not pattern:
        return None
    m = _space_aware_rx(pattern).search(text, start_pos)
    return (m.start(), m.end()) if m else None


# =========================================
#   ЯКОРЯ / ОКНА
# =========================================
def _anchor_candidates(region: str, pat: str, max_hits: int = 60):
    r"""Узкие якоря tolerant-поиском, плюс «сдвиг к началу слова» после пробела."""
    pat_head = pat[:14]
    anchors = []
    if len(pat_head) >= 3:
        anchors.append(pat_head)
    m = re.search(r"[A-Za-zА-Яа-я0-9]{4,}", pat)
    if m:
        anchors.append(m.group(0))
    m2 = re.search(r"[\(\)\[\]\{\}\^\+\-\=\/\\\|\*]{2,}", pat)
    if m2:
        anchors.append(m2.group(0))

    starts = set()
    for a in anchors:
        rx = _space_aware_rx(a)
        for hit in rx.finditer(region):
            starts.add(hit.start())
            if len(starts) >= max_hits:
                break
        if len(starts) >= max_hits:
            break

    if not starts:
        starts.add(0)

    more = []
    for s in list(starts):
        m3 = re.search(rf"(?<={_WS_CLASS})\S", region[s:])
        if m3:
            more.append(s + m3.start())
    starts.update(more)

    return sorted(starts)[:max_hits]


# =========================================
#   ПРИЁМОЧНЫЕ ПАРАМЕТРЫ
# =========================================
def _max_acceptable_score(L: int) -> float:
    if L <= 4:  return 0.0
    if L <= 7:  return 0.06
    if L <= 12: return 0.12
    return 0.18


# =========================================
#   ЛУЧШЕЕ СОВПАДЕНИЕ
# =========================================
@lru_cache(maxsize=4096)
def _best_fuzzy_span(text: str, start_pos: int, pattern: str, *, window: int = 280):
    """
    Возвращает dict {start,end,score} или None.
    Порядок: .find → tolerant-search → локальный .find → локальный tolerant → фаззи.
    Фикс: короткие алфавитно-цифровые паттерны (1–2 символа) не матчим «внутри» слова.
    """
    # --- локальные хелперы, чтобы не зависеть от внешних определений ---
    def _is_short_alnum_token(pat: str) -> bool:
        s = _norm(pat)
        return 1 <= len(s) <= 2 and s.isalnum()

    def _is_word_char(ch: str) -> bool:
        return bool(ch) and (ch.isalnum() or ch == "_")

    def _inside_word(txt: str, s: int, e: int) -> bool:
        left = txt[s - 1] if s > 0 else ""
        right = txt[e] if e < len(txt) else ""
        return _is_word_char(left) and _is_word_char(right)

    short_token = _is_short_alnum_token(pattern)

    pat_n = _norm(pattern)
    if not pat_n:
        return None

    # 1) Глобальный .find (с фильтром «внутри слова» для коротких токенов)
    idx = text.find(pattern, start_pos)
    if idx != -1 and not (short_token and _inside_word(text, idx, idx + len(pattern))):
        return {"start": idx, "end": idx + len(pattern), "score": 0.0}

    # 2) Глобальный tolerant-поиск (пропускаем попадания «внутри слова» для коротких токенов)
    rx = _space_aware_rx(pattern)
    m = rx.search(text, start_pos)
    while m and short_token and _inside_word(text, m.start(), m.end()):
        m = rx.search(text, m.end())
    if m:
        return {"start": m.start(), "end": m.end(), "score": 0.0}

    # Окно
    reg_len = max(window, int(3.5 * len(pat_n) + 40))
    region = text[start_pos:start_pos + reg_len]
    if not region:
        return None

    # 3) Локальный .find по окну (с фильтром границ)
    idx2 = region.find(pattern)
    if idx2 != -1:
        s = start_pos + idx2
        if not (short_token and _inside_word(text, s, s + len(pattern))):
            return {"start": s, "end": s + len(pattern), "score": 0.0}

    # 4) Локальный tolerant-поиск по окну (с пропуском «внутри слова»)
    m = rx.search(region)
    while m and short_token and _inside_word(text, start_pos + m.start(), start_pos + m.end()):
        m = rx.search(region, m.end())
    if m:
        s = start_pos + m.start()
        e = start_pos + m.end()
        return {"start": s, "end": e, "score": 0.0}

    # 5) Фаззи-поиск
    starts = _anchor_candidates(region, pattern, max_hits=60) or [0]
    L = max(1, len(pat_n))
    lengths = sorted({L, max(1, int(L * 0.85)), int(L * 1.2)})
    best = None
    best_score = 1e9
    accept = _max_acceptable_score(L)

    for i in starts:
        for ell in lengths:
            cand = region[i:i + ell]
            if not cand:
                continue
            s_abs = start_pos + i
            e_abs = s_abs + len(cand)
            # не берём кандидатов «внутри слова» для коротких токенов
            if short_token and _inside_word(text, s_abs, e_abs):
                continue
            score = lev(_norm(cand), pat_n)
            if (score < best_score - 1e-6
                or (abs(score - best_score) < 1e-6 and s_abs < (best["start"] if best else 10**9))):
                best_score = score
                best = {"start": s_abs, "end": e_abs, "score": score}
                if score == 0.0:
                    return best
        if best and best["score"] <= min(0.05, accept):
            return best

    if not best or best["score"] > accept:
        return None
    return best


# =========================================
#   СТЫКОВКА
# =========================================
def stitch_markers_fast(original_text: str, generated_text: str, neighbor_margin: float = 0.12) -> str:
    r"""
    Сопоставляет <w>...</w> из LLM-текста с ОРИГИНАЛОМ.
    Сохраняет ровно исходные символы оригинала (включая кратные/неразрывные пробелы, «мат.минус», и т.п.).
    """
    gen_spans = _extract_spans(generated_text)
    if not gen_spans:
        return original_text

    out, pos, i, n = [], 0, 0, len(gen_spans)
    while i < n:
        g = gen_spans[i]
        best_g = _best_fuzzy_span(original_text, pos, g)
        best_next = _best_fuzzy_span(original_text, pos, gen_spans[i + 1]) if i + 1 < n else None

        if (best_g and best_next
            and (best_next["score"] + neighbor_margin < best_g["score"])
            and (best_next["start"] <= best_g["start"])):
            out.append(original_text[pos:best_next["start"]])
            pos = best_next["start"]
            best_g = _best_fuzzy_span(original_text, pos, g)

        if not best_g:
            i += 1
            continue

        s, e = best_g["start"], best_g["end"]
        if s > pos:
            out.append(original_text[pos:s])
        out.append("<w>" + original_text[s:e] + "</w>")
        pos = e
        i += 1

    out.append(original_text[pos:])
    return "".join(out)

In [ ]:
# Для примера покажем, как работает стыковка
stitched = stitch_markers_fast(
    'Это  первоначальная строка без тегов.',
    'Эта <w>строка без тегов</w>'
)
print(stitched)

Это  первоначальная <w>строка без тегов</w>.


In [ ]:
#@title Маленький, умный, сложный парсер

def extract_tag_segments(text: str):
    """
    Принимает строку с <w>...</w> и возвращает список [ [start, end], ... ],
    где start и end — позиции внутри текста без тегов.
    Сами теги (<w>, </w>) не учитываются в подсчёте длины.
    """
    segments = []
    plain_pos = 0   # позиция в тексте без тегов

    # ВАЖНО: добавили альтернативу '|<' — чтобы буквальный символ '<' тоже учитывался как текст
    # Порядок альтернатив важен: сначала распознаём теги <w>/<\w>, потом текст без '<',
    # и только затем одиночный символ '<' как обычный текст.
    token_pattern = re.compile(r"<w>|</w>|[^<]+|<")

    active = False
    start_pos = None

    for m in token_pattern.finditer(text):
        token = m.group(0)
        if token == "<w>":
            active = True
            start_pos = plain_pos
        elif token == "</w>":
            if active and start_pos is not None and plain_pos > start_pos:
                segments.append([start_pos, plain_pos])
            active = False
            start_pos = None
        else:
            # обычный текст (включая одиночный '<'): увеличиваем позицию на его длину
            plain_pos += len(token)

    # Если был незакрытый <w>... в конце — считаем, что закрывается концом текста
    if active and start_pos is not None and plain_pos > start_pos:
        segments.append([start_pos, plain_pos])

    return segments

In [ ]:
# Покажем, как работает парсер
extract_tag_segments('Я <w>ел</w> пельмени <w>утром</w>')

[[2, 4], [14, 19]]

In [ ]:
for df in tqdm(dfs, desc='Парсинг отрезков'):
    for i in range(len(df)):
        # Qwen возвращает ответ с CoT. Нужно этот CoT обрезать и положить в переменную marked_solution
        text = df.loc[i, 'response_0']
        marked_solution = text[text.rfind('think') + 8:]

        # Оригинальное решение студента
        solution = df.loc[i, 'solution']

        # LLM могла вернуть текст решение не точь-в-точь.
        # Мы это исправляем, сопоставляет слова по расстоянию Левенштейна
        # Функция переносит маркеры <w>...</w> с ответа модели на оригинальное решение
        stitched_solution = stitch_markers_fast(
            solution, marked_solution
        )

        # Теперь при помощи несложного парсера преобразовываем строку с <w>...</w> в список полуинтервалов [[a, b)]
        tags = extract_tag_segments(stitched_solution)

        # Сохраняем ответ
        df.loc[i, 'answer'] = str(tags)

In [ ]:
# Посмотрим, все ли корректно обработалось?
dfs[0].head()

,idx,task,solution,preprocessed_solution,response_0,answer
0,0,"Докажите: если оператор диагонализируем, то ег...",Ситуация такая: у нас линейный оператор T на к...,Ситуация такая: у нас линейный оператор T на к...,"Хорошо, мне нужно проверить решение задачи и н...","[[211, 212], [571, 572], [1911, 1912], [2030, ..."
1,1,Вычисли $ \displaystyle \lim_{x \rightarrow 8}...,"Смотрю на предел при x, стремящемся к 8, у нас...","Смотрю на предел при x, стремящемся к 8, у нас...","Хорошо, давай разберусь с этой задачей. Мне ну...","[[523, 541], [604, 625], [759, 769], [784, 785..."
2,2,Пусть \(1\le p\le\infty\). Требуется установит...,Хотим показать: если в ℓ^p (при 1 ≤ p ≤ ∞) взя...,Хотим показать: если в l^p (при 1 <= p <= ∞) в...,"Хорошо, мне нужно проверить решение задачи о п...",[]
3,3,Вычисли $ \displaystyle \lim_{x \rightarrow -1...,"Посмотрим, что делает эта дробь при x, стремящ...","Посмотрим, что делает эта дробь при x, стремящ...","Хорошо, давай разберусь с этой задачей. Мне ну...","[[54, 55], [74, 75], [296, 297], [434, 437], [..."
4,4,Вычисли $ \displaystyle \lim_{x \rightarrow 8}...,"Смотрю на выражение при x, стремящемся к 8: вв...","Смотрю на выражение при x, стремящемся к 8: вв...","Хорошо, давай разберусь с этой задачей. Мне ну...","[[470, 471], [476, 477], [494, 495], [510, 511..."


In [ ]:
# Красиво проверим первую задачу
dfs[0]['answer'] = dfs[0]['answer'].apply(literal_eval) # чтобы в answer был список, а не строка
highlight_errors(dfs[0].loc[0, 'solution'], dfs[0].loc[0, 'answer'])

Отлично! парсинг прошел успешно. Теперь самая интересная часть архитектуры: **Ансамблирование**

### 🧬 **Этап 4: Ансамблирование и постобработка**

**Ансамблирование**

Предположим, у нас есть несколько моделей, каждая из которых сделала свои выводы о том, где находится ошибка:

![](https://github.com/droyti46/aiijc2025/blob/main/img/stage2/ensemble_1.png?raw=true)

Можно представить, что модель "отдаёт свой голос" за каждый символ, находящийся на отрезке, который она выделила:

![](https://github.com/droyti46/aiijc2025/blob/main/img/stage2/ensemble_2.png?raw=true)

Таким образом, можно варьировать порог `threshold` и выбирать, какие символы будут включены в финальный ответ:

![](https://github.com/droyti46/aiijc2025/blob/main/img/stage2/ensemble_3.png?raw=true)

Ансамблирование даёт **устойчивость** к «смещению границ» разных предсказаний

**Постобработка**

Иногда на размеченном привате встречались кейсы, когда в решении неправильный ответ повторяется несколько раз, но модель отмечает неверный ответ только один раз. Глупо терять на таком баллы, поэтому была разработана алгоритмическая постобработка

![](https://github.com/droyti46/aiijc2025/blob/main/img/stage2/postproseccing.png?raw=true)

Постобработка дополняет разметку `answer` по правилу «последнее ↔ предпоследнее число», аккуратно находя числа в тексте и сверяя их по значению.

Что именно делает:

* Парсит текущие интервалы `answer`.
* Находит **последнее число в конце решения**, обрезая «хвостовой мусор» (пробелы, точки, многоточия, закрывающие скобки/кавычки, тире и т.п.).
* Находит **предыдущее числовое слово** левее него. Регэксп специально устроен так, чтобы **не ловить** цифры внутри токенов вроде `2π`, `2x`, `3e` и т.д. (требуется правая не-словная граница).
* Нормализует оба токена (минусы `−–—` → `-`, убирает не-разрывные/тонкие пробелы, `,` → `.`) и сравнивает **по значению через SymPy** (поддерживаются целые, десятичные, простые дроби `p/q`, числа с разделителями тысяч).
* Если числа **эквивалентны** и **хотя бы одно** из них уже выделено (полностью или частично), то:

  * гарантирует **полное выделение последнего** числа;
  * добавляет/расширяет выделение у предпоследнего, **кроме** случая, когда у предпоследнего уже есть **только частичное** покрытие — тогда его **не расширяет**.
* Возвращает обновлённые интервалы, нормализованные к `int` и отсортированные.

Итог: если студент в конце решения повторяет ранее полученное числовое значение (например, «получили 5. Ответ: 5.»), разметка автоматически делает оба вхождения согласованно выделенными, но бережно относится к уже частично размеченным участкам.

Помимо этого постобработка исправляет некоторые ошибки, например, иногда парсер может случайно выделять одиночные русские символы и знаки препинания. Это тоже исправляется алгоритмически.

In [ ]:
#@title Ансамблер

# ---------- утилиты для работы с полуинтервалами ----------
def _to_spans(obj):
    """ Приводит answer к списку [[s,e), ...]; допускает строку с таким списком. """
    if obj is None or (isinstance(obj, float) and pd.isna(obj)):
        return []
    if isinstance(obj, str):
        try:
            obj = literal_eval(obj)
        except Exception:
            return []
    spans = []
    try:
        for p in obj:
            if p is None or len(p) != 2:
                continue
            s, e = int(p[0]), int(p[1])
            spans.append([s, e])
    except Exception:
        return []
    return spans

def spans_to_mask(spans, length):
    """ Из списка полуинтервалов -> булева маска длины length. Клипуем в [0, length]. """
    mask = np.zeros(length, dtype=np.uint8)
    for s, e in _to_spans(spans):
        s = max(0, min(length, s))
        e = max(0, min(length, e))
        if e > s:
            mask[s:e] = 1
    return mask.astype(bool)

def mask_to_spans(mask):
    """ Обратно: булева маска -> список полуинтервалов [[s,e), ...]. """
    if len(mask) == 0:
        return []
    arr = np.asarray(mask, dtype=np.uint8)
    padded = np.concatenate(([0], arr, [0]))
    diff = np.diff(padded)
    starts = np.where(diff == 1)[0]
    ends   = np.where(diff == -1)[0]
    return [[int(s), int(e)] for s, e in zip(starts, ends)]

# ---------- основной ансамблер ----------
def ensemble_answers_by_index(dfs, threshold=None):
    """
    Ансамблирование ответов по позициям (i-я строка каждого df -> голос для i-й строки первого df).

    Параметры:
      dfs       : список pd.DataFrame, dfs[0] обязан иметь колонки 'task' и 'solution'.
                  В каждом df колонка с голосом называется 'answer' (списки полуинтервалов).
      threshold : необязательный int — сколько голосов нужно, чтобы включить символ.
                  По умолчанию — строгое большинство (> N/2), то есть floor(N/2)+1.

    Возвращает:
      pd.DataFrame с колонками ['task', 'solution', 'answer'] длиной ровно как dfs[0].
    """
    if not dfs:
        raise ValueError("Нужен хотя бы один DataFrame.")
    # Работаем по позициям, поэтому сбрасываем индексы
    dfs = [d.reset_index(drop=True) for d in dfs]
    base = dfs[0]
    if 'task' not in base.columns or 'solution' not in base.columns:
        raise ValueError("Первый DataFrame должен содержать колонки 'task' и 'solution'.")

    n_models = len(dfs)
    need_votes = threshold if threshold is not None else (n_models // 2 + 1)  # строгое большинство

    # Подготавливаем списки ответов, обрезаем/дополняем до длины base
    base_len = len(base)
    answers_per_df = []
    for d in dfs:
        if 'answer' in d.columns:
            arr = d['answer'].tolist()
        else:
            arr = [[]] * len(d)
        if len(arr) < base_len:
            arr = arr + [[]] * (base_len - len(arr))
        else:
            arr = arr[:base_len]
        answers_per_df.append(arr)

    def vote_row(i, sol):
        L = len(sol) if isinstance(sol, str) else 0
        if L == 0:
            return []
        votes = np.zeros(L, dtype=np.uint16)
        for arr in answers_per_df:
            votes += spans_to_mask(arr[i], L).astype(np.uint16)
        return mask_to_spans(votes >= need_votes)

    out = base[['task', 'solution']].copy()
    out['answer'] = [vote_row(i, base.at[i, 'solution']) for i in range(base_len)]
    return out

In [ ]:
#@title Небольшая постобработка

# =========================
#   Регулярные выражения
# =========================

# Хвостовой «мусор» после числа: пробелы, точки, многоточия, закрывающие скобки/кавычки, тире и т.п.
TRAILING_JUNK_RE = re.compile(
    r'[\s\.\,\;\:\!\?\u2026\)\]\}\u00BB\u00AB"\'\u203A\u2039\u3001\uFF0C\u3002\u00B7\u2022\-\u2212–—]*\Z'
)

# Число в самом конце строки (ответ): знак, разделители тысяч, десятичная точка/запятая и простые дроби p/q.
NUM_AT_END_RE = re.compile(
    r'((?:[+\-−–—]\s*)?'                          # необязательный знак; пробел допустим только после него
    r'(?:\d{1,3}(?:[ \u00A0\u2009]\d{3})*|\d+)'   # целая часть (возможны разделители тысяч)
    r'(?:[.,]\d+)?'                               # дробная часть
    r'(?:/\d+(?:[.,]\d+)?)?'                      # простая дробь p/q (опционально)
    r')\s*\Z'
)

# Любое числовое «слово» внутри текста (для поиска предыдущих чисел).
# Важно: требуем правую "не-словную" границу (?!\w), чтобы НЕ ловить '2' в '2π', '2x', '3e', …
NUM_TOKEN_RE = re.compile(
    r'(?<![\w/\^])'                               # слева не буква/цифра, '/', '^'
    r'('
    r'[+\-−–—]?\s*'
    r'(?:\d{1,3}(?:[ \u00A0\u2009]\d{3})*|\d+)'   # целая часть (включая вариант с разделителями тысяч)
    r'(?:[.,]\d+)?'                               # дробная часть
    r'(?:/\d+(?:[.,]\d+)?)?'                      # простая дробь p/q
    r')'
    r'(?!\w)'                                     # правая граница: дальше не должно быть буквы/цифры/_
)

# =========================
#   Вспомогательные функции
# =========================

def _parse_spans(x):
    """Преобразует значение столбца `answer` в список интервалов.

    Поддерживает как уже-список (списки/кортежи с парами [s,e)),
    так и строку с литералом списка.

    Args:
      x: Значение ячейки `answer`.

    Returns:
      Список интервалов вида [[start, end], ...]. Невалидные записи отбрасываются.
    """
    if isinstance(x, str):
        try:
            return ast.literal_eval(x)
        except Exception:
            return []
    return list(x) if isinstance(x, (list, tuple)) else []


def _tighten_start(text: str, a: int, b: int):
    """Сдвигает левую границу вправо, убирая ведущие пробелы/переносы."""
    while a < b and text[a].isspace():
        a += 1
    return a, b


def _find_last_number_span(text: str):
    """Находит интервал последнего (самого правого) числа в тексте.

    Игнорирует хвостовой «мусор» (., …, скобки, кавычки, тире) после числа.

    Args:
      text: Полный текст решения.

    Returns:
      Кортеж (a, b) границ числа или None, если число в конце не найдено.
    """
    if not isinstance(text, str) or not text:
        return None
    m_tail = TRAILING_JUNK_RE.search(text)
    end_excl = m_tail.start() if m_tail else len(text)
    if end_excl <= 0:
        return None
    prefix = text[:end_excl]
    m_num = NUM_AT_END_RE.search(prefix)
    if not m_num:
        return None
    a, b = m_num.span(1)
    a, b = _tighten_start(text, a, b)
    return a, b


def _find_prev_number_span(text: str, before_idx: int):
    """Ищет предыдущее числовое «слово» левее позиции before_idx.

    Берём самое правое число в префиксе текста text[:before_idx],
    предварительно убрав хвостовой мусор в конце префикса.

    Args:
      text: Полный текст.
      before_idx: Правая граница поиска (исключительно).

    Returns:
      Кортеж (a, b) границ найденного числа или None.
    """
    prefix = text[:before_idx]
    m_tail = TRAILING_JUNK_RE.search(prefix)
    end2 = m_tail.start() if m_tail else len(prefix)
    prefix2 = prefix[:end2]
    last = None
    for m in NUM_TOKEN_RE.finditer(prefix2):
        last = m
    if not last:
        return None
    a, b = last.span(1)
    a, b = _tighten_start(text, a, b)
    return a, b


def _normalize_numeric_token(s: str) -> str:
    """Нормализует числовой токен для парсинга SymPy (минусы, пробелы, запятая→точка)."""
    t = s.strip()
    t = t.replace('\u2212', '-').replace('–', '-').replace('—', '-')
    t = t.replace('\u00A0', '').replace('\u2009', '').replace(' ', '')
    t = t.replace(',', '.')
    return t


def _to_sympy_value(s: str):
    """Преобразует строковое число к объекту SymPy или возвращает None при неудаче."""
    t = _normalize_numeric_token(s)
    try:
        return sp.nsimplify(t)
    except Exception:
        try:
            return sp.sympify(t)
        except Exception:
            return None


def _sympy_equal(a_str: str, b_str: str) -> bool:
    """Сравнивает два числовых токена на эквивалентность значений через SymPy."""
    va, vb = _to_sympy_value(a_str), _to_sympy_value(b_str)
    if va is None or vb is None:
        return False
    try:
        return sp.simplify(va - vb) == 0
    except Exception:
        return va == vb


def _coverage_status(spans, a: int, b: int) -> str:
    """Оценивает покрытие интервала [a,b) текущими выделениями.

    Args:
      spans: Список интервалов [[start, end], ...].
      a: Левая граница.
      b: Правая граница (не включительно).

    Returns:
      "none" (нет пересечений), "partial" (частичное покрытие), "full" (полное покрытие).
    """
    overlaps = []
    for s, e in spans:
        left, right = max(a, int(s)), min(b, int(e))
        if right > left:
            overlaps.append((left, right))
    if not overlaps:
        return "none"
    overlaps.sort()
    merged = [list(overlaps[0])]
    for x, y in overlaps[1:]:
        if x <= merged[-1][1]:
            merged[-1][1] = max(merged[-1][1], y)
        else:
            merged.append([x, y])
    covered_len = sum(r - l for l, r in merged)
    if covered_len >= (b - a):
        return "full"
    return "partial"


def _apply_last_prev_rule(solution_text: str, spans):
    """Правило «последнее ↔ предпоследнее число» с оговоркой про частичное покрытие.

    Если последние и предпоследние числа эквивалентны (через SymPy) и
    хотя бы одно уже выделено (частично или полностью), то выделяем оба,
    КРОМЕ случая: последнее уже выделено, а у предпоследнего выделена только часть —
    тогда предпоследнее не расширяем.
    """
    if not isinstance(solution_text, str):
        return spans

    last = _find_last_number_span(solution_text)
    if not last:
        return spans
    la, lb = last

    prev = _find_prev_number_span(solution_text, la)
    if not prev:
        return spans
    pa, pb = prev

    last_str = solution_text[la:lb]
    prev_str = solution_text[pa:pb]
    if not _sympy_equal(last_str, prev_str):
        return spans

    last_cov = _coverage_status(spans, la, lb)   # "none" | "partial" | "full"
    prev_cov = _coverage_status(spans, pa, pb)

    if last_cov == "none" and prev_cov == "none":
        return spans  # ни одно не выделено — ничего не делаем

    # 1) Последнее — стараемся иметь полное покрытие
    if last_cov != "full" and not any(s == la and e == lb for s, e in spans):
        spans.append([la, lb])

    # 2) Предпоследнее — не расширяем, если у него уже частичное покрытие и последнее выделено
    if not ( (last_cov in ("partial", "full")) and (prev_cov == "partial") ):
        if prev_cov != "full" and not any(s == pa and e == pb for s, e in spans):
            spans.append([pa, pb])

    return spans


# =========================
#   Чистка одиночных символов
# =========================

def _is_russian_letter(ch: str) -> bool:
    """True, если символ — русская буква (А–Я, а–я, Ё/ё)."""
    return ('А' <= ch <= 'Я') or ('а' <= ch <= 'я') or ch in ('Ё', 'ё')

def _is_punctuation(ch: str) -> bool:
    """True, если символ — знак препинания (Unicode-категория P*)."""
    try:
        return ud.category(ch).startswith('P')
    except Exception:
        return False

def _remove_single_char_noise(solution_text: str, spans):
    """Удаляет одиночные выделения, если это пунктуация или русская буква."""
    cleaned = []
    for a, b in spans:
        if b - a == 1:
            ch = solution_text[a:b]
            if _is_punctuation(ch) or _is_russian_letter(ch):
                continue  # выбрасываем одиночный шум
        cleaned.append([a, b])
    return cleaned


def process_row(row):
    """Постобработка строки DataFrame.

    Шаги:
      1) Применяем правило «последнее ↔ предпоследнее число» (с учетом частичных покрытий).
      2) Удаляем одиночные символы, если это пунктуация или русская буква.

    Args:
      row: Строка DataFrame (Series) с полями 'solution' и 'answer'.

    Returns:
      Список интервалов `answer` после постобработки.
    """
    solution_text = row.get('solution', '')
    spans = _parse_spans(row.get('answer', []))
    spans = _apply_last_prev_rule(solution_text, spans)
    spans = [[int(a), int(b)] for a, b in spans]
    spans.sort(key=lambda ab: (ab[0], ab[1]))
    spans = _remove_single_char_noise(solution_text, spans)
    return spans

In [ ]:
from google.colab import files

# Производим ансамблирование всех ответов моделей с порогом = 2
submission = ensemble_answers_by_index(dfs, threshold=2)

# Применяем постобработку
submission['answer'] = submission.apply(process_row, axis=1)

# Сохраняем предсказание
submission.to_csv('submission.csv', index=False)

# Автоматическое скачивание на компьютер проверяющего
files.download('submission.csv')

# На всякий случай, если сабмит нужен вам конкретно в /content
submission.to_csv('/content/submission.csv', index=False)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
# Убедимся, что в сабмите действительно 400 строк
print(f'В сабмите {len(submission)} строк')

В сабмите 400 строк


In [ ]:
# Для проверки посмотрим, как ансамбль разметил первую задачу
highlight_errors(submission.loc[0, 'solution'], submission.loc[0, 'answer'])

# 🎉 **Завершение**

Спасибо большое **Сберу** за органицазию и **Центральному Университету** за предоставленный кейс. Было очень интересно и весело поучаствовать в конкурсе =)

С уважением,\
Команда **"Сидим не рыпаемся"**

<br>

<p align="center">
  <img src="https://github.com/droyti46/aiijc2025/blob/main/img/snr-logo.png?raw=true" width="500">
</p>